![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

#  Introduction to Text Mining and Natural Language Processing
## Homework 1: Research Project about hotel prices on Booking.com

GROUP 11: Luis Francisco Alvarez Poli, Vanessa Kromm, Clarice Mottet

In [1]:
import json
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
import re
from sklearn.linear_model import LinearRegression
from sklearn import metrics

path_in_ = r'C:\Users\clari\OneDrive\Desktop\VSCode\Term2_IntroLanguage\homework1\github_repo\Booking_Scraping'
path_out_ = r'C:\Users\clari\OneDrive\Desktop\VSCode\Term2_IntroLanguage\homework1\outputs'
path_out_reg_ = r'C:\Users\clari\OneDrive\Desktop\VSCode\Term2_IntroLanguage\homework1\github_repo\Booking_Scraping\data_for_regression'


In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path, firefox_binary_path, download=True):
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder, download)
    options.binary_location = firefox_binary_path  # Set Firefox binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    
    # Enter the website address here
    browser.get(link)
    
    time.sleep(5)  
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [4]:
# lets open booking:

dfolder='C:/Users/vanes/Downloads'
geko_path='C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/TA-Sessions/geckodriver.exe'
link='https://www.booking.com/index.html?lang=en'
firefox_binary_path = r'C:\Program Files\Mozilla Firefox\firefox.exe'  


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path, firefox_binary_path=firefox_binary_path)

### close cookies and google pop-up

In [5]:
browser.find_element(by='xpath',value='//*[@id="onetrust-accept-btn-handler"]').click()

In [6]:
####### works only somethimes, don't know why##################
browser.find_element(By.XPATH, '//*[@id="close"]').click()


NoSuchElementException: Message: Unable to locate element: //*[@id="close"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


## Input the place and dates

### First we will look at Barcelona for the time of the event

In [9]:
# input the place
def input_place(place):
    browser.find_element(by='xpath',value='//*[@id=":re:"]').click()
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)


In [8]:
# input the dates
def input_dates(from_day, to_day, change_month = 0):
    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

    browser.find_element('css selector',css).click()

    # click to change the month to May/June
    if change_month == 1:
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button/span/span').click()
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]').click()
        browser.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]').click()

    # week from Monday to Sunday covering the whole length of Primavera Sound
    path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'

    dates = browser.find_elements('xpath', path)

    for date in dates:
        date_value = date.get_attribute("data-date")
        
        if date_value == from_day:
            date.click()
        elif date_value == to_day:
            date.click()
            break

## Iterate through all pages and hotels to extract the information

In [10]:
# find total number of pages
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[7]/button')
    return(int(a[-1].text))


In [11]:
css_pages = 'div.b16a89683f:nth-child(3) > button:nth-child(1) > span:nth-child(1) > span:nth-child(1)'
def get_information(pages):
    # Get the original window handle
    hotel_names = []
    ratings = []
    room_descriptions = []
    prices = []
    location_descriptions = []
    hotel_links = []

    # beach_distances = []
    
    sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
    for hotel in sections:
        hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text

    for page in range(int(pages)+1):    
    #for page in range(int(2)+1):      
        #Print page that it is in 
        print(f'Page: {page + 1}')
        sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
        for hotel in sections:
            hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text
            hotel_names.append(hotel_name)
            # extract ratings
            try:
                rating = hotel.find_element('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]').text
            except:
                rating = np.nan
            ratings.append(rating)    
            # extract prices
            try:
                price = hotel.find_element('xpath','.//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]').text
            except:
                price = np.nan
            prices.append(price) 
            # extract room_descriptions
            try:
                room_description = hotel.find_element('xpath','.//div[@class="c19beea015"]').text
            except:
                room_description = np.nan
            room_descriptions.append(room_description)
            # extract location_descriptsion
            try:
                location_description = hotel.find_element('xpath', './/div[@class="abf093bdfe ecc6a9ed89"]').text
            except:
                location_description = np.nan  
            location_descriptions.append(location_description)
            # get links of all hotels
            try:
                hotel_link = hotel.find_element('xpath', './/a[@class="a78ca197d0"]').get_attribute('href')
            except:
                hotel_link = np.nan  
            hotel_links.append(hotel_link)

        next_page = browser.find_element('css selector', css_pages).click()
        time.sleep(2)        
       

    df = pd.DataFrame({'hotel_name': hotel_names, 'rating': ratings, 'room_description': room_descriptions, 'price': prices, 'location_descirption': location_descriptions, 'hotel_link': hotel_links})
    return df

In [12]:
input_place('Barcelona')
input_dates('2024-05-27', '2024-06-02', 1)
# click on "search"
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath')    

In [14]:
# close "Genius Pop up"
browser.find_element(By.XPATH, '/html/body/div[51]/div/div/div/div[1]/div[1]/div/button/span/span').click()

In [41]:
pages = get_number_pages(browser)
barcelona_treatment = get_information(pages)

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40


In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract text from a given URL using BeautifulSoup and XPath-like expression
def extract_text_from_url(url, expression):
    # Fetch the HTML content
    response = requests.get(url)
    html_content = response.content

    # Parse HTML with BeautifulSoup using 'lxml' parser
    soup = BeautifulSoup(html_content, 'lxml')

    # Use a CSS selector (similar to XPath) to find the desired element
    element = soup.select_one(expression)

    # Extract text from the found element
    if element:
        return element.get_text(strip=True)
    else:
        return None


# XPath-like expression for the element you want to extract
expression_to_extract = '#summary'

# Extract text from each URL in the DataFrame and store in a new column
barcelona_treatment['long_description'] = barcelona_treatment['hotel_link'].apply(lambda url: extract_text_from_url(url, expression_to_extract))


In [44]:
barcelona_treatment.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_treatment.csv')

In [2]:
barcelona_treatment = pd.read_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_treatment.csv')

In [7]:
barcelona_treatment.describe()

,Unnamed: 0,rating
count,1002.000000,974.000000
mean,500.500000,8.132957
std,289.396786,0.893503
min,0.000000,1.000000
25%,250.250000,7.800000
50%,500.500000,8.300000
75%,750.750000,8.675000
max,1001.000000,10.000000


### now we have to change the date to extract the hotel prices for the week before the event

We only scrape the name and the price here because rating and description will not change.

In [45]:
css_pages = 'div.b16a89683f:nth-child(3) > button:nth-child(1) > span:nth-child(1) > span:nth-child(1)'
def get_only_prices(pages):
    # Get the original window handle
    hotel_names = []
    prices_week_before = []
    
    sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
    for hotel in sections:
        hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text

    for page in range(int(pages)+1):    
    #for page in range(int(1)+1):      
        #Print page that it is in 
        print(f'Page: {page + 1}')
        sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
        for hotel in sections:
            hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text
            hotel_names.append(hotel_name)  
            # extract prices
            try:
                price_week_before = hotel.find_element('xpath','.//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]').text
            except:
                price_week_before = np.nan
            prices_week_before.append(price_week_before)

        next_page = browser.find_element('css selector', css_pages).click()
        time.sleep(2)        

    df = pd.DataFrame({'hotel_name': hotel_names, 'price_week_before': prices_week_before})
    return df

In [46]:
input_dates('2024-05-20', '2024-05-26')
# click on "search"
my_xpath='/html/body/div[4]/div/div[2]/div/div[1]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath') 
pages = get_number_pages(browser)
barcelona_control = get_only_prices(pages)

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41


In [49]:
barcelona_control.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_control.csv')

In [22]:
barcelona_control = pd.read_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_control.csv')

Creating a dataframe that contains prices for both weeks

In [51]:
barcelona_merged = pd.merge(barcelona_treatment, barcelona_control, on='hotel_name', how='left')

barcelona_merged = barcelona_merged.dropna(subset=['price_week_before'])


In [53]:
barcelona_merged.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_merged.csv')

### now we have to change the city to extract the hotel prices for the week before the event for Rome

In [54]:
# clear the field first
browser.find_element(by='xpath',value='//*[@id=":re:"]').clear()
input_place('Rome')
time.sleep(3)
browser.find_element(by='xpath',value='//*[@id="autocomplete-result-1"]').click()
# add sleep time, because otherwise it will still show results for Barcelona
time.sleep(3)
# click on "search"
my_xpath='/html/body/div[4]/div/div[2]/div/div[1]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath') 
pages = get_number_pages(browser)
rome_control = get_only_prices(pages)

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41


In [55]:
rome_control.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_control.csv')

### extract prices and all the information for Rome during the week of event

In [57]:
input_dates('2024-05-27', '2024-06-02')
my_xpath='/html/body/div[4]/div/div[2]/div/div[1]/div/form/div[1]/div[4]/button/span'
check_and_click(browser,my_xpath , type='xpath') 
pages = get_number_pages(browser)
rome_treatment = get_information(pages)

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41


In [58]:
# Extract text from each URL in the DataFrame and store in a new column
rome_treatment['long_description'] = rome_treatment['hotel_link'].apply(lambda url: extract_text_from_url(url, expression_to_extract))

In [60]:
rome_treatment.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_treatment.csv')

In [18]:
rome_control = pd.read_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_control.csv')

In [62]:
rome_merged = pd.merge(rome_treatment, rome_control, on='hotel_name', how='left')

rome_merged = rome_merged.dropna(subset=['price_week_before'])


In [64]:
rome_merged.to_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_merged.csv')

In [22]:
barcelona_merged=pd.read_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_merged.csv')
rome_merged=pd.read_csv('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_merged.csv')

To run the regression, the columns including the price have to be integers

In [23]:
barcelona_merged['price'] = barcelona_merged['price'].str.replace('€', '').str.replace(',', '').astype(int)
rome_merged['price'] = rome_merged['price'].str.replace('€', '').str.replace(',', '').astype(int)
barcelona_merged['price_week_before'] = barcelona_merged['price_week_before'].str.replace('€', '').str.replace(',', '').astype(int)
rome_merged['price_week_before'] = rome_merged['price_week_before'].str.replace('€', '').str.replace(',', '').astype(int)

In [36]:
barcelona_merged.dtypes

Unnamed: 0                int64
hotel_name               object
rating                  float64
room_description         object
price                     int32
location_descirption     object
long_description         object
Unnamed: 0_y            float64
price_week_before         int32
dtype: object

### Possible features based on the description

From the location description it is possible to extract the distance to the city centre

In [24]:
def extract_distance_city_centre(df):    
    pattern = r'(\d?\.?\d,?\d*?\s\w+)'

    # Apply the regex pattern to the 'location' column and create a new 'distance' column
    df['distance_city_centre'] = df['location_descirption'].str.extract(pattern)

    # regex patterns to capture kilometers and meters
    pattern_km = r'(\d?\.?\d,?\d*?)\s?km'
    pattern_m = r'(\d+)\s?m'

    # Apply the regex patterns to the 'distance' column and create new 'distance_meter' and 'distance_meter_only' columns
    df['distance_city_centre_meter'] = df['distance_city_centre'].str.extract(pattern_km)
    df['distance_city_centre_meter_only'] = df['distance_city_centre'].str.extract(pattern_m)

    # Convert the 'distance_city_centre_meter' column to meters (assuming 1 km = 1000 meters)
    df['distance_city_centre_meter'] = df['distance_city_centre_meter'].astype(float) * 1000

    # Replace NaN values in 'distance_city_centre_meter' with values from 'distance_city_centre_meter_only'
    df['distance_city_centre_meter'].fillna(df['distance_city_centre_meter_only'], inplace=True)

    # Drop the 'distance_city_centre_meter_only' column
    df.drop(columns=['distance_city_centre_meter_only'], inplace=True)

    # Convert the 'distance_city_centre_meter' column to integers
    df['distance_city_centre_meter'] = df['distance_city_centre_meter'].astype(int)
    df = df.drop('distance_city_centre', axis=1)
    return df


In [25]:
barcelona_merged = extract_distance_city_centre(barcelona_merged)

Since the distance sometimes is measured in km and sometimes in m, we will convert all the km to m and make it integers

In [27]:
rome_merged = extract_distance_city_centre(rome_merged)

In [29]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()

def strip(word):
    mod_string = re.sub(r'\W+', '', word)
    return mod_string

def pipeline_DTM(filepath, column_name):
    # Load csv file
    df = pd.read_csv(filepath)

    tokenized_texts = []
    # Iterate through the 'text' column and save each sentence as a string
    for index, row in df.iterrows():
        tokens = nltk.word_tokenize(str(row[column_name]))
        tokenized_texts.append(tokens)

    # lowercasing, stopword removal, and stemming
    corpus_stop = []
    corpus_stem = []
    for words in tokenized_texts:
        lowered_removed_stopwords = [abbr_or_lower(strip(w)) for w in words if
                                     abbr_or_lower(strip(w)) not in stopwords.words('english')]
        #print(lowered_removed_stopwords)
        corpus_stop.append(lowered_removed_stopwords)
        stemmed = [SnowballStemmer("english").stem(w) for w in lowered_removed_stopwords]
        # print(stemmed)
        corpus_stem.append(" ".join(stemmed))

    # TF-IDF Vectorization
    tfidf = TfidfVectorizer(ngram_range=(1, 2), norm=None, min_df=0.05, max_df=0.8)
    tfidf_vectorized_text = tfidf.fit_transform(corpus_stem)
    tfidf_vectorized_text = tfidf_vectorized_text.todense()

    # Creating a DataFrame from the TF-IDF vectorized data
    df_tfidf_vectorized = pd.DataFrame(tfidf_vectorized_text, columns=tfidf.get_feature_names_out())

    return df_tfidf_vectorized


In [31]:
df_tfidf_barcelona = pipeline_DTM('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/barcelona_merged.csv','long_description')

In [32]:
# Sum the TF-IDF scores for each term across all documents
term_scores = df_tfidf_barcelona.sum(axis=0)

# Convert the result to a DataFrame for better visualization
df_term_scores = pd.DataFrame({'Term': term_scores.index, 'TF-IDF Score': term_scores.values})

# Sort the DataFrame by TF-IDF score in descending order
df_term_scores = df_term_scores.sort_values(by='TF-IDF Score', ascending=False)

# Display the top N terms
top_terms = 30  # Set the number of top terms you want to display
print(f"Top {top_terms} Terms:")
print(df_term_scores.head(top_terms))

Top 30 Terms:
           Term  TF-IDF Score
53        apart   2392.482131
218       hotel   1917.729335
232          km   1840.066413
356        room   1502.052565
443        walk   1306.909416
137          de   1212.064499
275       minut   1147.169977
335    properti   1105.033135
294       offer   1083.719552
268        metr   1052.143558
45      airport   1009.634432
65         away   1000.456467
409     station    907.704223
204      gracia    901.986313
178      featur    883.366006
276  minut walk    863.438119
346      rambla    854.109360
271       metro    847.177884
255       locat    829.343916
312     passeig    827.088091
313  passeig de    821.101100
49         also    807.894581
223      includ    802.770514
139   de gracia    798.483348
332      privat    784.637174
207       guest    784.132388
62        avail    758.880555
436          tv    748.923674
112   catalunya    748.894302
241          la    735.533069


possible features: control whether hotel or apartment, distance to airport

In [33]:
df_tfidf_rome = pipeline_DTM('C:/Users/vanes/Desktop/BSE/Term 2/Introduction to Text Mining and NLP/HW1/rome_merged.csv','long_description')

In [34]:
# Sum the TF-IDF scores for each term across all documents
term_scores = df_tfidf_rome.sum(axis=0)

# Convert the result to a DataFrame for better visualization
df_term_scores = pd.DataFrame({'Term': term_scores.index, 'TF-IDF Score': term_scores.values})

# Sort the DataFrame by TF-IDF score in descending order
df_term_scores = df_term_scores.sort_values(by='TF-IDF Score', ascending=False)

# Display the top N terms
top_terms = 30  # Set the number of top terms you want to display
print(f"Top {top_terms} Terms:")
print(df_term_scores.head(top_terms))

Top 30 Terms:
              Term  TF-IDF Score
82           apart   1461.057981
455        station   1215.483977
313          metro   1138.358513
314  metro station   1120.837404
230          guest    902.429786
242           hous    793.341121
232     guest hous    702.826189
405           room    643.037562
504        vatican    638.900716
498           unit    614.554566
56        accommod    613.812590
206         featur    579.312155
308           metr    576.873020
373         privat    511.857635
350          peter    495.312414
115            bed    487.447249
353         piazza    485.288810
197          equip    476.463154
241           home    475.625199
453             st    475.099907
454       st peter    472.683906
92            area    472.163969
384         provid    465.863637
248         includ    463.589094
101           away    456.644231
215     flatscreen    439.265467
216  flatscreen tv    438.995257
397           roma    438.269370
98           avail    437.006

Cleaning the data scraped to prepare it for regression analysis.



In [2]:
#path to retrieve the data pulled in

os.listdir(path_in_)

dict_import = {}
list_import = []
for file in os.listdir(path_in_):
    if 'csv' in file:
        file_str = file[:-4]
        dict_import[file_str] = pd.read_csv(path_in_ + '\\' + file, dtype = 'str')
        dict_import[file_str].columns = dict_import[file_str].columns.str.strip().str.lower().str.replace(' ','_')
        dict_import[file_str] = dict_import[file_str].apply(lambda x: x.str.strip())
        dict_import[file_str].drop(columns = ['unnamed:_0'], inplace = True)
        dict_import[file_str]['file_source'] = file
        list_import.append(dict_import[file_str])
        print(file_str+'--------')
        print(dict_import[file_str].columns)

df_all = pd.concat(list_import, ignore_index = True)
df_all.to_excel(path_out_ + '\\' + 'df_all_data.xlsx')

#removing these because I'll clean things first before the merge
del dict_import['barcelona_merged']
del dict_import['rome_merged']


barcelona_control--------
Index(['hotel_name', 'price_week_before', 'file_source'], dtype='object')
barcelona_merged--------
Index(['hotel_name', 'rating', 'room_description', 'price',
       'location_descirption', 'hotel_link', 'long_description',
       'price_week_before', 'file_source'],
      dtype='object')
barcelona_treatment--------
Index(['hotel_name', 'rating', 'room_description', 'price',
       'location_descirption', 'hotel_link', 'long_description',
       'file_source'],
      dtype='object')
rome_control--------
Index(['hotel_name', 'price_week_before', 'file_source'], dtype='object')
rome_merged--------
Index(['hotel_name', 'rating', 'room_description', 'price',
       'location_descirption', 'hotel_link', 'long_description',
       'price_week_before', 'file_source'],
      dtype='object')
rome_treatment--------
Index(['hotel_name', 'rating', 'room_description', 'price',
       'location_descirption', 'hotel_link', 'long_description',
       'file_source'],
      dty

In [ ]:
#make copy of raw imports
df_b_control = dict_import['barcelona_control'].copy()
df_b_treat = dict_import['barcelona_treatment'].copy()
df_r_control = dict_import['rome_control'].copy()
df_r_treat = dict_import['rome_treatment'].copy()

#add in city field
df_b_control['city'] = 'barcelona'
df_b_treat['city'] = 'barcelona'
df_r_control['city'] = 'rome'
df_r_treat['city'] = 'rome'

#append control and treatment together for consistent cleaning practices
df_control = pd.concat([df_b_control, df_r_control], ignore_index = True)
df_treat = pd.concat([df_b_treat, df_r_treat], ignore_index = True)
df_control.reset_index(drop = True, inplace = True)
df_treat.reset_index(drop = True, inplace = True)

#remove duplicates

#first by hotel_link
df_treat.drop_duplicates(subset = ['hotel_link'], inplace = True)
#then by hotel_name and price
df_treat.drop_duplicates(subset = ['hotel_name','price'], inplace = True)

#if a hotel has the same name and same room description but different prices
#remove from the data as price is not consistent for same room
df_treat['count'] = 1
df_treat['count'] = df_treat.groupby(by = ['hotel_name','room_description'])['count'].transform('sum')
df_treat = df_treat[df_treat['count']==1]
df_treat.drop(columns = ['count'], inplace = True)

#reset index after removing duplicates
df_control.reset_index(drop = True, inplace = True)
df_treat.reset_index(drop = True, inplace = True)

# df_control.to_excel(path_out_+'\\'+'df_control.xlsx', index = False)
# df_treat.to_excel(path_out_+'\\'+'df_treat.xlsx', index = False)

#convert ratings and price to numeric
df_control['price_week_before'] = df_control['price_week_before'].str.replace('€','').str.replace(',','').str.strip()
df_control['price_week_before'] = pd.to_numeric(df_control['price_week_before'])

df_treat['price'] = df_treat['price'].str.replace('€','').str.replace(',','').str.strip()
df_treat['price'] = pd.to_numeric(df_treat['price'])

df_treat['rating'] = pd.to_numeric(df_treat['rating'])


In [21]:
def common_terms_cleaning_map(str_):
    #function created to condense terms that are similar

    #assume no cleaning unless otherwise
    str_clean = str_

    #common terms cleaning
    if str_.find('left_at_this_price') > -1:
        str_clean = 'few_left_at_this_price'
    return str_clean

In [22]:
#ended up not using
def meters_squared(str_):
    value_ = np.nan
    if str_.find('m²') > -1:
        value_ = float(str_.replace('m²',''))
    return value_

In [23]:
#This code was written formulaically in excel based on 12 bed descriptions that needed special
#attention, it tooks less than 2 minutes to complete even though it seems long

def manual_beds(df_beds):

    #manual code written in excel
    df_beds.loc[df_beds['description']=='multiple_bed_types','extract_number_of_beds'] = 0
    df_beds.loc[df_beds['description']=='2_beds_(1_double,_1_large_double)','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='double_room_with_extra_bed','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='economy_double_sofa_bed','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='small_double_room_with_french_bed','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='6-bed_mixed_dormitory_room','extract_number_of_beds'] = 6
    df_beds.loc[df_beds['description']=='deluxe_double_room_with_extra_bed','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='double_or_twin_room_with_extra_bed','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='double_room_with_french_bed','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='double_room_with_queen_bed_-_basement','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='quadruple_bedbunks_room_with_bathroom','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='standard_triple_room_with_one_double_bed_and_one_single_bed_-_non-smoking','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='superior_double_or_twin_room_with_extra_bed_(2_adults_+_1_child)','extract_number_of_beds'] = 2
    df_beds.loc[df_beds['description']=='twin_room_with_bunk_beds_and_shared_bathroom','extract_number_of_beds'] = 1
    df_beds.loc[df_beds['description']=='multiple_bed_types','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='2_beds_(1_double,_1_large_double)','number_of_double_beds'] = 2
    df_beds.loc[df_beds['description']=='double_room_with_extra_bed','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='economy_double_sofa_bed','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='small_double_room_with_french_bed','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='6-bed_mixed_dormitory_room','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='deluxe_double_room_with_extra_bed','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='double_or_twin_room_with_extra_bed','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='double_room_with_french_bed','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='double_room_with_queen_bed_-_basement','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='quadruple_bedbunks_room_with_bathroom','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='standard_triple_room_with_one_double_bed_and_one_single_bed_-_non-smoking','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='superior_double_or_twin_room_with_extra_bed_(2_adults_+_1_child)','number_of_double_beds'] = 1
    df_beds.loc[df_beds['description']=='twin_room_with_bunk_beds_and_shared_bathroom','number_of_double_beds'] = 0
    df_beds.loc[df_beds['description']=='multiple_bed_types','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='2_beds_(1_double,_1_large_double)','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_extra_bed','number_of_single_beds'] = 1
    df_beds.loc[df_beds['description']=='economy_double_sofa_bed','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='small_double_room_with_french_bed','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='6-bed_mixed_dormitory_room','number_of_single_beds'] = 6
    df_beds.loc[df_beds['description']=='deluxe_double_room_with_extra_bed','number_of_single_beds'] = 2
    df_beds.loc[df_beds['description']=='double_or_twin_room_with_extra_bed','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_french_bed','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_queen_bed_-_basement','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='quadruple_bedbunks_room_with_bathroom','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='standard_triple_room_with_one_double_bed_and_one_single_bed_-_non-smoking','number_of_single_beds'] = 1
    df_beds.loc[df_beds['description']=='superior_double_or_twin_room_with_extra_bed_(2_adults_+_1_child)','number_of_single_beds'] = 1
    df_beds.loc[df_beds['description']=='twin_room_with_bunk_beds_and_shared_bathroom','number_of_single_beds'] = 0
    df_beds.loc[df_beds['description']=='multiple_bed_types','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='2_beds_(1_double,_1_large_double)','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_extra_bed','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='economy_double_sofa_bed','number_of_sofa_beds'] = 1
    df_beds.loc[df_beds['description']=='small_double_room_with_french_bed','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='6-bed_mixed_dormitory_room','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='deluxe_double_room_with_extra_bed','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='double_or_twin_room_with_extra_bed','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_french_bed','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_queen_bed_-_basement','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='quadruple_bedbunks_room_with_bathroom','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='standard_triple_room_with_one_double_bed_and_one_single_bed_-_non-smoking','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='superior_double_or_twin_room_with_extra_bed_(2_adults_+_1_child)','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='twin_room_with_bunk_beds_and_shared_bathroom','number_of_sofa_beds'] = 0
    df_beds.loc[df_beds['description']=='multiple_bed_types','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='2_beds_(1_double,_1_large_double)','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_extra_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='economy_double_sofa_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='small_double_room_with_french_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='6-bed_mixed_dormitory_room','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='deluxe_double_room_with_extra_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='double_or_twin_room_with_extra_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_french_bed','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='double_room_with_queen_bed_-_basement','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='quadruple_bedbunks_room_with_bathroom','number_of_bunk_beds'] = 2
    df_beds.loc[df_beds['description']=='standard_triple_room_with_one_double_bed_and_one_single_bed_-_non-smoking','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='superior_double_or_twin_room_with_extra_bed_(2_adults_+_1_child)','number_of_bunk_beds'] = 0
    df_beds.loc[df_beds['description']=='twin_room_with_bunk_beds_and_shared_bathroom','number_of_bunk_beds'] = 1
    return df_beds

In [24]:
#extract the bed counts

def extract_bed_counts(df_beds):

    for i in range(len(df_beds['description'])):

        #extract_number_of_beds
        list_bed = [r"(\d+)\s*_beds"]
        for extract_bed_regex in list_bed:
            if re.findall(extract_bed_regex, df_beds.loc[i,'description']):
                beds = int(re.findall(extract_bed_regex, df_beds.loc[i,'description'])[0])
                df_beds.loc[i, 'extract_number_of_beds'] = beds

        #extract number of double beds
        list_double_bed = [r"(\d+)\s*_(large_)?double_bed",r"(\d+)\s*_extra-large_double",r"(\d+)\s*_doubles\b",r"(\d+)\s*_large_double\b",r"(\d+)\s*_double\b",r"(\d+)\s*_large_doubles\b"]
        for double_bed_regex in list_double_bed:
            if re.findall(double_bed_regex, df_beds.loc[i,'description']):
                beds = int(re.findall(double_bed_regex, df_beds.loc[i,'description'])[0][0])
                df_beds.loc[i, 'number_of_double_beds'] += beds

        #extract number of single beds
        list_single_bed = [r"(\d+)\s*_single"]
        for single_bed_regex in list_single_bed:
            if re.findall(single_bed_regex, df_beds.loc[i,'description']):
                beds = int(re.findall(single_bed_regex, df_beds.loc[i,'description'])[0])
                df_beds.loc[i, 'number_of_single_beds'] = beds

        #extract number of sofa beds
        list_sofa = [r"(\d+)\s*_sofa_bed",r"(\d+)\s*_futon"]
        for sofa_bed_regex in list_sofa:
            if re.findall(sofa_bed_regex, df_beds.loc[i,'description']):
                beds = int(re.findall(sofa_bed_regex, df_beds.loc[i,'description'])[0][0])
                df_beds.loc[i, 'number_of_sofa_beds'] += beds

        #extract number of bunk beds
        list_bunk_bed = [r"(\d+)\s*_bunk_bed"]
        for bunk_bed_regex in list_bunk_bed:
            if re.findall(bunk_bed_regex, df_beds.loc[i,'description']):
                beds = int(re.findall(bunk_bed_regex, df_beds.loc[i,'description'])[0][0])
                df_beds.loc[i, 'number_of_bunk_beds'] = beds

    return df_beds


In [25]:
#define and extract characteristics by the below
#string split by \n
#string split by '•'

#extract distinct characteristics
list_room_desc = []
for i in range(len(df_treat['hotel_name'])):
    list_split_line = df_treat.loc[i,'room_description'].split('\n')
    for str_ in list_split_line:
        #dot condition
        if '•' in str_:
            list_split_dot = str_.split('•')
            for str_dot in list_split_dot:
                str_dot = str_dot.strip().lower().replace(' ','_')
                str_ = common_terms_cleaning_map(str_)
                list_room_desc.append(str_dot)
        else:
            str_ = str_.strip().lower().replace(' ','_')
            str_ = common_terms_cleaning_map(str_)
            list_room_desc.append(str_)

df_room = pd.DataFrame(list_room_desc, columns = ['description'])
df_room['freq'] = 1
df_room['freq'] = df_room.groupby(by = ['description'])['freq'].transform('sum')
df_room.drop_duplicates(subset = ['description'], inplace = True)
df_room.sort_values(by = ['freq','description'], ascending = [False, True], inplace = True)
df_room.reset_index(drop = True, inplace = True)

df_room['bed'] = 0

for i in range(len(df_room['freq'])):
    str_ = df_room.loc[i, 'description']
    if str_.find('bed') > -1:
        df_room.loc[i,'bed'] = 1
    if str_.find('bedroom') > -1:
        df_room.loc[i,'bed'] = 0
        
#-----------------------------------------------------
#extract bed information

#define what specifics we will use for beds
room_list = list(df_room[df_room['bed']==1]['description'])

#create dataframe for bed extraction
df_beds = pd.DataFrame(room_list, columns=['description'])
df_beds['extract_number_of_beds'] = 0
df_beds['number_of_double_beds'] = 0
df_beds['number_of_single_beds'] = 0
df_beds['number_of_sofa_beds'] = 0
df_beds['number_of_bunk_beds'] = 0

#using function created using regex
df_beds = extract_bed_counts(df_beds)
df_beds = manual_beds(df_beds)

df_beds['total_number_of_beds'] = df_beds['number_of_double_beds'] + df_beds['number_of_single_beds'] + df_beds['number_of_sofa_beds'] + df_beds['number_of_bunk_beds']

df_room = df_room.merge(df_beds, how = 'left', on = ['description'])

# df_room.to_excel(path_out_+'\\'+'df_room.xlsx')


In [26]:
#compile characteristics to be used for regression

#create dataframe that's distinct by hotel
df_treat_desc = pd.DataFrame()

#create map for hotel characteristics for each hotel
for i in range(len(df_treat['hotel_name'])):
    list_room_desc = []
    list_split_line = df_treat.loc[i,'room_description'].split('\n')
    for str_ in list_split_line:
        #dot condition
        if '•' in str_:
            list_split_dot = str_.split('•')
            for str_dot in list_split_dot:
                str_dot = str_dot.strip().lower().replace(' ','_')
                str_ = common_terms_cleaning_map(str_)
                list_room_desc.append(str_dot)
        else:
            str_ = str_.strip().lower().replace(' ','_')
            str_ = common_terms_cleaning_map(str_)
            list_room_desc.append(str_)
    df_ = pd.DataFrame(list_room_desc, columns = ['description'])
    df_['hotel_name'] = df_treat.loc[i, 'hotel_name']
    df_treat_desc = pd.concat([df_treat_desc, df_], ignore_index = True)
    df_treat_desc.reset_index(drop = True)

#map on the bed information
df_treat_desc = df_treat_desc.merge(df_beds, how = 'left', on = ['description'])
df_treat_desc.drop(columns = ['extract_number_of_beds'], inplace = True)

#one hot encoding
list_one_hot = ['few_left_at_this_price',
                'free_cancellation',
                'breakfast_included']
df_treat_desc['few_left_at_this_price'] = 0
df_treat_desc['free_cancellation'] = 0
df_treat_desc['breakfast_included'] = 0

for col in list_one_hot:
    df_treat_desc.loc[df_treat_desc['description']==col, col] = 1

numeric_cols = ['number_of_double_beds',
       'number_of_single_beds', 'number_of_sofa_beds', 'number_of_bunk_beds',
       'total_number_of_beds', 'few_left_at_this_price', 'free_cancellation',
       'breakfast_included']

for col in numeric_cols:
    df_treat_desc[col] = df_treat_desc.groupby(by = ['hotel_name'])[col].transform('max')

df_treat_desc.drop(columns = ['description'], inplace = True)
df_treat_desc.drop_duplicates(subset = ['hotel_name'], inplace = True)

#if we don't know total_beds, we're going to use the average
fillna_zero = ['number_of_double_beds',
       'number_of_single_beds', 'number_of_sofa_beds', 'number_of_bunk_beds']
for col in fillna_zero:
    df_treat_desc[col] = df_treat_desc[col].fillna(0)

df_treat_desc.loc[df_treat_desc['total_number_of_beds']==0, 'total_number_of_beds'] = np.nan
most_frequent_beds = df_treat_desc['total_number_of_beds'].mode()[0]

df_treat_desc['total_number_of_beds'] = df_treat_desc['total_number_of_beds'].fillna(most_frequent_beds)


In [41]:
#create the dataframes for the regression

#create a ratings map
df_ratings = df_treat[['hotel_name','city','rating']].copy()
df_ratings['rating'] = df_ratings.groupby(by = ['city'])['rating'].transform(lambda x: x.fillna(x.mean()))
df_ratings.drop(columns = ['city'], inplace = True)

#determine info we have for both hotels
common_hotels = df_control.merge(df_treat, on='hotel_name', how='inner')['hotel_name']

#update df_control and df_treat to only include the common hotels
df_control = df_control[df_control['hotel_name'].isin(common_hotels)]
df_treat = df_treat[df_treat['hotel_name'].isin(common_hotels)]

df_control.reset_index(drop = True, inplace = True)
df_treat.reset_index(drop = True, inplace = True)

df_control = df_control[['city','hotel_name','price_week_before']]
df_treat = df_treat[['city','hotel_name','price']]

df_control = df_control.merge(df_ratings, how = 'left', on = ['hotel_name'])
df_control = df_control.merge(df_treat_desc, how = 'left', on = ['hotel_name'])

df_treat = df_treat.merge(df_ratings, how = 'left', on = ['hotel_name'])
df_treat = df_treat.merge(df_treat_desc, how = 'left', on = ['hotel_name'])

df_control.to_excel(path_out_reg_ + '\\' + 'df_control_clean.xlsx', index = False)
df_treat.to_excel(path_out_reg_ + '\\' + 'df_treat_clean.xlsx', index = False)


## Regressions related to task 6 (three equations)

In [4]:
df_control = pd.read_excel('C:/Users/vanes/Git_repositories/Booking_Scraping/data_for_regression/df_control_clean.xlsx')
df_treatment = pd.read_excel('C:/Users/vanes/Git_repositories/Booking_Scraping/data_for_regression/df_treat_clean.xlsx')

In [ ]:
df_control = pd.read_excel('/Users/lfap/Documents/BSE/T2/NLP/Intro/Booking/Booking_Scraping/data_for_regression/df_control_clean.xlsx')
df_treatment = pd.read_excel('/Users/lfap/Documents/BSE/T2/NLP/Intro/Booking/Booking_Scraping/data_for_regression/df_treat_clean.xlsx')

NameError: name 'pd' is not defined

In [5]:
df_treatment.head()


,city,hotel_name,price,rating,number_of_double_beds,number_of_single_beds,number_of_sofa_beds,number_of_bunk_beds,total_number_of_beds,few_left_at_this_price,free_cancellation,breakfast_included
0,barcelona,Ciudad Condal Hostal - Paseo de Gracia,1058,7.200000,0,2,0,0,2,1,0,0
1,barcelona,DO Plaça Reial powered by Sonder,2923,8.400000,1,0,0,0,1,1,0,0
2,barcelona,SEALONA VILA OLIMPICA,543,7.974017,1,0,0,0,1,1,1,0
3,barcelona,Hotel Cantón,903,8.300000,0,2,0,0,2,1,1,0
4,barcelona,Catalonia Eixample 1864,1329,8.400000,0,2,0,0,2,0,1,0


In [10]:
df_control.head()

,city,hotel_name,price_week_before,rating,number_of_double_beds,number_of_single_beds,number_of_sofa_beds,number_of_bunk_beds,total_number_of_beds,few_left_at_this_price,free_cancellation,breakfast_included
0,barcelona,Ciudad Condal Hostal - Paseo de Gracia,996,7.200000,0,2,0,0,2,1,0,0
1,barcelona,DO Plaça Reial powered by Sonder,2478,8.400000,1,0,0,0,1,1,0,0
2,barcelona,SEALONA VILA OLIMPICA,596,7.974017,1,0,0,0,1,1,1,0
3,barcelona,Catalonia Sagrada Familia,970,8.200000,0,2,0,0,2,0,1,0
4,barcelona,Catalonia Atenas,1019,8.200000,0,2,0,0,2,0,1,0


In [6]:
df_control_reg = df_control.copy()
df_treat_reg = df_treatment.copy()

df_treat_reg = df_treat_reg[['hotel_name','price','city']]
df_control_reg = df_control_reg[['hotel_name','price_week_before','city']]
df_control_reg.head()

,hotel_name,price_week_before,city
0,Ciudad Condal Hostal - Paseo de Gracia,996,barcelona
1,DO Plaça Reial powered by Sonder,2478,barcelona
2,SEALONA VILA OLIMPICA,596,barcelona
3,Catalonia Sagrada Familia,970,barcelona
4,Catalonia Atenas,1019,barcelona


In [7]:
df_treat_reg['time_treatment'] = 1
df_control_reg['time_treatment'] = 0
df_treat_reg['city_treatment'] = (df_treat_reg['city'] == 'barcelona').astype(int)
df_control_reg['city_treatment'] = (df_control_reg['city'] == 'barcelona').astype(int)

df_treat_reg.head()



,hotel_name,price,city,time_treatment,city_treatment
0,Ciudad Condal Hostal - Paseo de Gracia,1058,barcelona,1,1
1,DO Plaça Reial powered by Sonder,2923,barcelona,1,1
2,SEALONA VILA OLIMPICA,543,barcelona,1,1
3,Hotel Cantón,903,barcelona,1,1
4,Catalonia Eixample 1864,1329,barcelona,1,1


In [8]:
df_control_reg.head()

,hotel_name,price_week_before,city,time_treatment,city_treatment
0,Ciudad Condal Hostal - Paseo de Gracia,996,barcelona,0,1
1,DO Plaça Reial powered by Sonder,2478,barcelona,0,1
2,SEALONA VILA OLIMPICA,596,barcelona,0,1
3,Catalonia Sagrada Familia,970,barcelona,0,1
4,Catalonia Atenas,1019,barcelona,0,1


In [9]:
df_control_reg = df_control_reg.rename(columns={'price_week_before': 'price'})

df_combined_reg = pd.concat([df_treat_reg, df_control_reg], ignore_index=True)

df_combined_reg.shape

(2981, 5)

In [10]:
df_combined_reg = df_combined_reg[['price','time_treatment','city_treatment']]
df_combined_reg.iloc[[3,895,1509,2418]]

,price,time_treatment,city_treatment
3,903,1,1
895,3158,1,0
1509,1314,0,1
2418,704,0,0


<h3> Define the variables for the DiD and run the regressions

In [11]:
x1 = df_combined_reg['time_treatment'].to_frame()
x2 = df_combined_reg['city_treatment'].to_frame()
x1x2 = x1 * x2

y = df_combined_reg['price']



In [12]:
# Preparing the data for simple regressions
x1 = df_combined_reg[['time_treatment']]  # For the first simple regression
x2 = df_combined_reg[['city_treatment']]  # For the second simple regression

# Preparing the data for DiD regression
x1x2 = df_combined_reg['time_treatment'] * df_combined_reg['city_treatment']
X_did = pd.concat([df_combined_reg[['time_treatment', 'city_treatment']], x1x2.rename('interaction')], axis=1)

# The dependent variable
y = df_combined_reg['price']

# Initialize the Linear Regression model
model = LinearRegression(fit_intercept=True)

# Run the first simple regression
model.fit(x1, y)
print("Simple Regression 1 (time_treatment): Coef =", model.coef_, "Intercept =", model.intercept_)

# Run the second simple regression
model.fit(x2, y)
print("Simple Regression 2 (city_treatment): Coef =", model.coef_, "Intercept =", model.intercept_)

# Run the DiD regression
model.fit(X_did, y)
print("DiD Regression: Coefs =", model.coef_, "Intercept =", model.intercept_)

Simple Regression 1 (time_treatment): Coef = [149.52965675] Intercept = 1713.8555334658713
Simple Regression 2 (city_treatment): Coef = [675.75642361] Intercept = 1448.7940780619092
DiD Regression: Coefs = [-25.62007054 501.05722718 355.11523703] Intercept = 1461.50066755674


In [13]:
import statsmodels.api as sm

# Simple regression with time-variable
# Add a constant term to the independent variable for the intercept
x1_with_const = sm.add_constant(x1)

# Create and fit the linear regression model
model = sm.OLS(y, x1_with_const).fit()

# Display the regression results summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     11.85
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           0.000583
Time:                        23:56:06   Log-Likelihood:                -25328.
No. Observations:                2981   AIC:                         5.066e+04
Df Residuals:                    2979   BIC:                         5.067e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1713.8555     30.519     56.

In [14]:
import statsmodels.api as sm

# Simple regression with city-variable
# Add a constant term to the independent variable for the intercept
x2_with_const = sm.add_constant(x2)

# Create and fit the linear regression model
model = sm.OLS(y, x2_with_const).fit()

# Display the regression results summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     262.4
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           1.25e-56
Time:                        23:57:22   Log-Likelihood:                -25208.
No. Observations:                2981   AIC:                         5.042e+04
Df Residuals:                    2979   BIC:                         5.043e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1448.7941     29.542     49.

In [15]:
# Add a constant term to the independent variables for the intercept
X_did_with_const = sm.add_constant(X_did)

# Create and fit the linear regression model with interaction term
model_interaction = sm.OLS(y, X_did_with_const).fit()

# Display the regression results summary
print(model_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     98.94
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           4.78e-61
Time:                        23:58:54   Log-Likelihood:                -25192.
No. Observations:                2981   AIC:                         5.039e+04
Df Residuals:                    2977   BIC:                         5.042e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1461.5007     41.404     35.

## Regressions related to task 7 
(addition of controls to the time period and city regression)

Initially this formula represented that given inputs (before, after) and (barcelona, rome) we are predicting the typical hotel price. The coefficients represent the average change we see between before/after and barcelona/rome.

We are going to be adding in inputs (rating, total_number_of_beds, few_left_at_this_price, free_cancellation) to see if we can determine whether these factors have an effect from one time period to another and 

In [7]:
#import the tables into the programming environment
df_control = pd.read_excel(path_out_reg_ + '\\' + 'df_control_clean.xlsx')
df_treat = pd.read_excel(path_out_reg_ + '\\' + 'df_treat_clean.xlsx')

df_control.rename(columns = {'price_week_before':'price'}, inplace = True)
df_control['timeperiod'] = 0
df_treat['timeperiod'] = 1

#create a dataframe for the regressions
df_reg = pd.concat([df_control, df_treat], ignore_index = False)
df_reg.reset_index(drop = True, inplace = True)

df_reg['city_'] = 1
df_reg.loc[df_reg['city']=='rome', 'city_'] = 0
df_reg.drop(columns = ['city'], inplace = True)
df_reg.rename(columns = {'city_':'city'}, inplace = True)

df_reg.drop(columns = ['hotel_name'], inplace = True)

#double check the first three regression models

#model1-----------------------------
X = df_reg[['timeperiod']].copy()
y = df_reg['price'].copy()
#create model
model1_timeperiod = LinearRegression()
model1_timeperiod.fit(X, y)
#get intercept and coefficients
intercept_timeperiod = model1_timeperiod.intercept_
coef_timeperiod = model1_timeperiod.coef_[0]
#print results
print("Model1", "Coefficient for timeperiod:", coef_timeperiod.round(2), "intercept:", intercept_timeperiod.round(2))

#model2-----------------------------
X = df_reg[['city']].copy()
y = df_reg['price'].copy()
#create model
model2_city = LinearRegression()
model2_city.fit(X, y)
#get intercept and coefficients
intercept_city = model2_city.intercept_
coef_city = model2_city.coef_[0]
#print results
print("Model2", "Coefficient for city:", coef_city.round(2), "intercept:", intercept_city.round(2))

#model3-----------------------------
#create interaction term
df_reg['timeperiod_city_interaction'] = df_reg['timeperiod'] * df_reg['city']

#Independent variables including the interaction term
X = df_reg[['timeperiod', 'city', 'timeperiod_city_interaction']].copy()
y = df_reg['price'].copy()
#Initialize and fit the linear regression model
model3_interaction = LinearRegression()
model3_interaction.fit(X, y)
#Coefficients
intercept_interaction = model3_interaction.intercept_
coef_timeperiod_interaction, coef_city_interaction, coef_interaction = model3_interaction.coef_
print(f"Model3 Coefficient for timeperiod: {coef_timeperiod_interaction.round(2)}, Coefficient for city: {coef_city_interaction.round(2)}, Coefficient for interaction: {coef_interaction.round(2)}, Intercept: {intercept_interaction.round(2)}")



Model1 Coefficient for timeperiod: 149.53 intercept: 1713.86
Model2 Coefficient for city: 675.76 intercept: 1448.79
Model3 Coefficient for timeperiod: -25.62, Coefficient for city: 501.06, Coefficient for interaction: 355.12, Intercept: 1461.5


IGNORE ALL THE FIGURES THEY HAVEN'T BEEN UPDATED

Dicussion of the first three equations:

In Model 1 we're looking at both the barcelona and rome population to understand how the price of hotels changes from the week before the Primavera Sound event and the week during the event. We're seeing the overall average price of hotels the week before the Primavera Sound event is approximately 1713 as given by the intercept of the equation. The following week has an average hotel price increase of approximately 150 given by the coefficient of our dummy treatment variable. This tells us that for both barcelona and rome, we see an increase in price from the week before the Primavera Sound event to the week during.

In Model 2 we're looking at both weeks to understand how the price of hotels differs between barcelona and rome. With our dummy city variable = 1 if rome and 0 otherwise, we see that the average price of a hotel stay for a week in barcelina is approximately 2125 as given by the intercept term. We're seeing that the average price of a hotel stay for a week in rome is on average approximately 676 less than what a stay in barcelona would be as given by the coefficient of the dummy city term.

In Model 3, we've created an interaction term specifically for the case where we're looking to see the hotel price rate change for the case where a person is in rome and during the event. First we look at the intercept which has the value 1962 representing the average stay in a hotel in barcelona the week before the Primavera Event. For the same week before the Primavera Event, we see that a stay in rome would be on average -501 less than in barcelona at approximately 1461.5 (1962.56 - 501.06) before the event. For the week during the event we see that the average price in barcelona increases by 329.5 as indicated by the time period coefficient. This marks a 16% increase between the two weeks in barcelona. The average price in rome goes from 1461.5 before the event to (1962 + -355.12 (rome and time period change)) equaling 1606.88. This relates to an increase in price of 145.38 (1606.88 - 1461.5) between the two weeks in rome resulting in an approx 10% price increase. While we cannot attribute the increase in price in barcelona to the event, we can say that hotel prices do increase in barcelona between the two weeks whereas we do not see the same magnitude of price increase in another european travel destination.

Equation 3 details the difference in difference analysis that we're looking for because it enables us to look at the percent increase in price between the time period in question and have an analogous city to compare it to. In the first two equations we were unable to determine how to attribute a change in rate depending on city or time period, but in equation two, we're able to look at the averages of the 4 cases: (before event, barcelona), (during event, barcelona), (before event, rome) and (during event, rome) corresponding to zeros and ones with the dummy variables to conduct a hotel price behavior analysis.